In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from matplotlib import pyplot
from sklearn.ensemble import RandomForestRegressor

In [2]:
df=pd.read_csv('covid19_us_county.csv')

In [3]:
df.head()

,date,county,state,fips,state_fips,county_fips,cases,deaths,new_day_cases,new_day_deaths,cases_per_capita_100k,deaths_per_capita_100k,new_day_cases_per_capita_100k,new_day_deaths_per_capita_100k,county_pop_2019_est,pop_per_sq_mile_2010
0,2020-01-21,Snohomish,Washington,53061,53,61,1,0,0.0,0.0,0.121642,0.0,0.0,0.0,822083.0,341.8
1,2020-01-22,Snohomish,Washington,53061,53,61,1,0,0.0,0.0,0.121642,0.0,0.0,0.0,822083.0,341.8
2,2020-01-23,Snohomish,Washington,53061,53,61,1,0,0.0,0.0,0.121642,0.0,0.0,0.0,822083.0,341.8
3,2020-01-24,Cook,Illinois,17031,17,31,1,0,0.0,0.0,0.019417,0.0,0.0,0.0,5150233.0,5495.1
4,2020-01-24,Snohomish,Washington,53061,53,61,1,0,0.0,0.0,0.121642,0.0,0.0,0.0,822083.0,341.8


In [4]:
df.corr()

,fips,state_fips,county_fips,cases,deaths,new_day_cases,new_day_deaths,cases_per_capita_100k,deaths_per_capita_100k,new_day_cases_per_capita_100k,new_day_deaths_per_capita_100k,county_pop_2019_est,pop_per_sq_mile_2010
fips,1.000000,0.999977,0.162142,-0.002563,0.000240,-0.007420,-0.002588,-0.051706,-0.077140,-0.018514,-0.039076,-0.071140,0.012924
state_fips,0.999977,1.000000,0.155417,-0.002379,0.000370,-0.007213,-0.002441,-0.051706,-0.077116,-0.018545,-0.039072,-0.070850,0.012532
county_fips,0.162142,0.155417,1.000000,-0.027147,-0.018797,-0.031243,-0.021645,-0.008212,-0.015738,0.001564,-0.006772,-0.053427,0.058815
cases,-0.002563,-0.002379,-0.027147,1.000000,0.962769,0.684344,0.654908,0.279070,0.391182,0.056632,0.125077,0.426104,0.659499
deaths,0.000240,0.000370,-0.018797,0.962769,1.000000,0.572143,0.603750,0.197639,0.345649,0.031846,0.095192,0.344259,0.639713
new_day_cases,-0.007420,-0.007213,-0.031243,0.684344,0.572143,1.000000,0.809226,0.196222,0.225665,0.142973,0.148591,0.471790,0.660430
new_day_deaths,-0.002588,-0.002441,-0.021645,0.654908,0.603750,0.809226,1.000000,0.150872,0.229655,0.052185,0.193259,0.341845,0.591228
cases_per_capita_100k,-0.051706,-0.051706,-0.008212,0.279070,0.197639,0.196222,0.150872,1.000000,0.589595,0.334430,0.245691,0.065798,0.126295
deaths_per_capita_100k,-0.077140,-0.077116,-0.015738,0.391182,0.345649,0.225665,0.229655,0.589595,1.000000,0.101620,0.359096,0.111462,0.203646
new_day_cases_per_capita_100k,-0.018514,-0.018545,0.001564,0.056632,0.031846,0.142973,0.052185,0.334430,0.101620,1.000000,0.098427,0.016780,0.038092


In [5]:
df.columns

Index(['date', 'county', 'state', 'fips', 'state_fips', 'county_fips', 'cases',
       'deaths', 'new_day_cases', 'new_day_deaths', 'cases_per_capita_100k',
       'deaths_per_capita_100k', 'new_day_cases_per_capita_100k',
       'new_day_deaths_per_capita_100k', 'county_pop_2019_est',
       'pop_per_sq_mile_2010'],
      dtype='object')

In [6]:
X=df[['fips', 'state_fips', 'county_fips',
       'deaths', 'new_day_cases', 'new_day_deaths', 'cases_per_capita_100k',
       'deaths_per_capita_100k', 'new_day_cases_per_capita_100k',
       'new_day_deaths_per_capita_100k', 'county_pop_2019_est',
       'pop_per_sq_mile_2010']]
y=df['cases']

In [ ]:
model = XGBClassifier()
model.fit(X, y)
# feature importance
print(model.feature_importances_)
# plot
pyplot.bar(range(len(model.feature_importances_)), model.feature_importances_)
pyplot.show()

In [8]:
rf = RandomForestRegressor()
rf.fit(X, y)
print ("Features sorted by their score:")
print (sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), X.columns), 
             reverse=True))

Features sorted by their score:
[(0.9409, 'deaths'), (0.0146, 'cases_per_capita_100k'), (0.0108, 'deaths_per_capita_100k'), (0.0083, 'new_day_cases'), (0.0062, 'county_fips'), (0.0049, 'pop_per_sq_mile_2010'), (0.0039, 'state_fips'), (0.0034, 'fips'), (0.0034, 'county_pop_2019_est'), (0.0015, 'new_day_deaths_per_capita_100k'), (0.0013, 'new_day_deaths'), (0.0007, 'new_day_cases_per_capita_100k')]


In [26]:
df=pd.read_csv('covid_npis_clean.csv')
df.head()

,Unnamed: 0,fip_code,county,state,npi,start_date,1/22/20,1/23/20,1/24/20,1/25/20,...,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20
0,0,1017,Chambers,Alabama,social_distancing,4/3/20,0,0,0,0,...,314,316,319,324,324,326,326,328,328,329
1,1,1017,Chambers,Alabama,shelter_in_place,4/4/20,0,0,0,0,...,314,316,319,324,324,326,326,328,328,329
2,2,1017,Chambers,Alabama,gathering_size_500_to_101,3/13/20,0,0,0,0,...,314,316,319,324,324,326,326,328,328,329
3,3,1017,Chambers,Alabama,gathering_size_100_to_26,3/16/20,0,0,0,0,...,314,316,319,324,324,326,326,328,328,329
4,4,1017,Chambers,Alabama,gathering_size_25_to_11,3/19/20,0,0,0,0,...,314,316,319,324,324,326,326,328,328,329


In [27]:
len(df)

2554

In [28]:
cols = df.keys()
confirmed = df.loc[:, cols[6]:cols[-1]]
confirmed=confirmed.replace(0,1)
confirmed.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20
0,1,1,1,1,1,1,1,1,1,1,...,314,316,319,324,324,326,326,328,328,329
1,1,1,1,1,1,1,1,1,1,1,...,314,316,319,324,324,326,326,328,328,329
2,1,1,1,1,1,1,1,1,1,1,...,314,316,319,324,324,326,326,328,328,329
3,1,1,1,1,1,1,1,1,1,1,...,314,316,319,324,324,326,326,328,328,329
4,1,1,1,1,1,1,1,1,1,1,...,314,316,319,324,324,326,326,328,328,329


In [29]:
confirmed=np.log(confirmed)
confirmed.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.749393,5.755742,5.765191,5.780744,5.780744,5.786897,5.786897,5.793014,5.793014,5.796058
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.749393,5.755742,5.765191,5.780744,5.780744,5.786897,5.786897,5.793014,5.793014,5.796058
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.749393,5.755742,5.765191,5.780744,5.780744,5.786897,5.786897,5.793014,5.793014,5.796058
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.749393,5.755742,5.765191,5.780744,5.780744,5.786897,5.786897,5.793014,5.793014,5.796058
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.749393,5.755742,5.765191,5.780744,5.780744,5.786897,5.786897,5.793014,5.793014,5.796058


In [38]:
dates=confirmed.keys()
dates = np.array([i for i in range(len(dates))]).reshape(-1, 1)

In [34]:
new_york=confirmed.iloc[1694].to_list()
new_york

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.3862943611198906,
 2.3978952727983707,
 2.3978952727983707,
 2.4849066497880004,
 2.9444389791664403,
 3.2188758248682006,
 4.007333185232471,
 4.553876891600541,
 5.0369526024136295,
 5.594711379601839,
 5.594711379601839,
 6.137727054086234,
 6.70196036600254,
 7.534228326274089,
 8.282483003730562,
 8.546946149565585,
 8.92665032079394,
 9.175127616220438,
 9.417760962825156,
 9.609384912281424,
 9.790094865181038,
 9.99300827847454,
 10.149292386393418,
 10.301457978882032,
 10.427268887462334,
 10.530842092546065,
 10.671719014219342,
 10.767199954191426,
 10.855319158329374,
 10.953592137139195,
 11.055735390370147,
 11.120638147127769,
 11.186932132354066,
 11.249949013154295,
 11.31206919673496,
 11.373985184938093,
 11.4

In [32]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [39]:
dates

array([[  0],
       [  1],
       [  2],
       [  3],
       [  4],
       [  5],
       [  6],
       [  7],
       [  8],
       [  9],
       [ 10],
       [ 11],
       [ 12],
       [ 13],
       [ 14],
       [ 15],
       [ 16],
       [ 17],
       [ 18],
       [ 19],
       [ 20],
       [ 21],
       [ 22],
       [ 23],
       [ 24],
       [ 25],
       [ 26],
       [ 27],
       [ 28],
       [ 29],
       [ 30],
       [ 31],
       [ 32],
       [ 33],
       [ 34],
       [ 35],
       [ 36],
       [ 37],
       [ 38],
       [ 39],
       [ 40],
       [ 41],
       [ 42],
       [ 43],
       [ 44],
       [ 45],
       [ 46],
       [ 47],
       [ 48],
       [ 49],
       [ 50],
       [ 51],
       [ 52],
       [ 53],
       [ 54],
       [ 55],
       [ 56],
       [ 57],
       [ 58],
       [ 59],
       [ 60],
       [ 61],
       [ 62],
       [ 63],
       [ 64],
       [ 65],
       [ 66],
       [ 67],
       [ 68],
       [ 69],
       [ 70],
      

In [35]:
new_york

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.3862943611198906,
 2.3978952727983707,
 2.3978952727983707,
 2.4849066497880004,
 2.9444389791664403,
 3.2188758248682006,
 4.007333185232471,
 4.553876891600541,
 5.0369526024136295,
 5.594711379601839,
 5.594711379601839,
 6.137727054086234,
 6.70196036600254,
 7.534228326274089,
 8.282483003730562,
 8.546946149565585,
 8.92665032079394,
 9.175127616220438,
 9.417760962825156,
 9.609384912281424,
 9.790094865181038,
 9.99300827847454,
 10.149292386393418,
 10.301457978882032,
 10.427268887462334,
 10.530842092546065,
 10.671719014219342,
 10.767199954191426,
 10.855319158329374,
 10.953592137139195,
 11.055735390370147,
 11.120638147127769,
 11.186932132354066,
 11.249949013154295,
 11.31206919673496,
 11.373985184938093,
 11.4

In [40]:
X = dates
X = sm.add_constant(X)


In [41]:
mod = sm.OLS(new_york, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     794.9
Date:                Tue, 19 May 2020   Prob (F-statistic):           9.69e-54
Time:                        13:12:53   Log-Likelihood:                -244.40
No. Observations:                 118   AIC:                             492.8
Df Residuals:                     116   BIC:                             498.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2629      0.354     -6.388      0.0

In [42]:
np.exp(-2.2629)

0.1040483067189046

In [43]:
np.exp(0.1475)

1.1589332848297509

In [44]:
def linear_predictions(t):
    return np.exp(-2.2629) * np.exp(0.1475) ** t

In [51]:
data=pd.DataFrame(columns=['new york cases actual'])
data['dates']=dates
data['new york cases actual']=new_york

Exception: Data must be 1-dimensional

In [50]:
data['Predictions'] = data.Time.apply(linear_predictions)
data.head(10)

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [14]:
df=pd.read_csv('covid_npis_dummy.csv')
df.head(20)

,Unnamed: 0,fip_code,county,state,start_date,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,npi_gathering_size_100_to_26,npi_gathering_size_10_0,npi_gathering_size_25_to_11,npi_gathering_size_500_to_101,npi_lockdown,npi_non-essential_services_closure,npi_religious_gatherings_banned,npi_school_closure,npi_shelter_in_place,npi_social_distancing
0,0,1017,Chambers,Alabama,4/3/20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1017,Chambers,Alabama,4/4/20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,1017,Chambers,Alabama,3/13/20,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,1017,Chambers,Alabama,3/16/20,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,1017,Chambers,Alabama,3/19/20,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,5,1017,Chambers,Alabama,3/28/20,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,6,1017,Chambers,Alabama,4/4/20,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,7,1017,Chambers,Alabama,3/20/20,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,8,1017,Chambers,Alabama,3/28/20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,1017,Chambers,Alabama,3/28/20,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [17]:
len(df)

2554